# Assignment 1
## problem statement 1
API Data Retrieval and Storage: You are tasked with fetching data from an external REST API, storing it in a local SQLite database, and displaying the retrieved data. The API provides a list of books in JSON format with attributes like title, author, and publication year.

In [4]:
%pip install requests

Note: you may need to restart the kernel to use updated packages.


### Loading API

In [5]:
import requests
import json
res = requests.get("https://openlibrary.org/search.json?q=the+lord+of+the+rings")
print(res.status_code)
data = res.json()
print(data)

200
{'numFound': 874, 'start': 0, 'numFoundExact': True, 'num_found': 874, 'documentation_url': 'https://openlibrary.org/dev/docs/api/search', 'q': 'the lord of the rings', 'offset': None, 'docs': [{'author_key': ['OL26320A'], 'author_name': ['J.R.R. Tolkien'], 'cover_edition_key': 'OL51694024M', 'cover_i': 14625765, 'ebook_access': 'borrowable', 'edition_count': 251, 'first_publish_year': 1954, 'has_fulltext': True, 'ia': ['lordofrings00tolk_5', 'yzklerinefendisi0000jrrt', 'hobbitguildpubli0000jrrt', 'lordofrings0000tolk_k7n1', 'lordofrings0000jrrt', 'lordofrings0000tolk_o3c6', 'lordofrings0000tolk_n6w1', 'osenordosaneis0000tolk', 'lordofrings0000tolk_d5h0', 'lordofrings00tolk_2', 'isbn_9780618343997', 'snj.lordofrings0000jrrt_y7c7', 'lordofringstrilo00jrrt', 'derherrderringed0000jrrt', 'lordofrings0000tolk_b2r2', 'lordofringsfello0000jrrt', 'isbn_9027456836', 'lordofringsbcaed0000jrrt', 'lordofrings0000tolk_y0v8', 'lordofrings00tolk_3', 'isbn_9780618640157', 'lordofrings00tolk_1', 'l

In [6]:
type(data)

dict

### Extracting books information like title, author and published year

In [7]:
docs = data['docs']


def extract_book_info(documents):
  book_data = []

  for doc in documents:

    title = doc.get('title', 'N/A')
    author_names = doc.get('author_name')
    # Join author names with a comma if it's a list, otherwise default to 'N/A'
    author = ', '.join(author_names) if isinstance(author_names, list) else 'N/A'
    first_publish_year = doc.get('first_publish_year', 'N/A')

    book_data.append({
        'title': title,
        'author_name': author,
        'first_publish_year': first_publish_year
    })
  return book_data


all_books = extract_book_info(docs)
print(f"Total books found: {len(all_books)}")
for book in all_books[:10]:
  print(book)


Total books found: 100
{'title': 'The Lord of the Rings', 'author_name': 'J.R.R. Tolkien', 'first_publish_year': 1954}
{'title': 'The Two Towers', 'author_name': 'J.R.R. Tolkien', 'first_publish_year': 1954}
{'title': 'The Fellowship of the Ring', 'author_name': 'J.R.R. Tolkien', 'first_publish_year': 1954}
{'title': 'The Return of the King', 'author_name': 'J.R.R. Tolkien', 'first_publish_year': 1950}
{'title': 'Novels (Hobbit / Lord of the Rings)', 'author_name': 'J.R.R. Tolkien', 'first_publish_year': 1979}
{'title': 'The Hobbit', 'author_name': 'J.R.R. Tolkien', 'first_publish_year': 1937}
{'title': 'Official the lôrd of the Rîngs 2022 Calendar', 'author_name': 'The Lord Of The Rings Publishing', 'first_publish_year': 2021}
{'title': 'The Lord of the Rings', 'author_name': 'Chris Conkling, Peter S. Beagle', 'first_publish_year': 1979}
{'title': 'The Return of the Shadow', 'author_name': 'J.R.R. Tolkien', 'first_publish_year': 1988}
{'title': 'Bored of the Rings', 'author_name': 'He

### Converting all the list of books data to DataFrame to use it in SQlite Database

In [10]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [11]:
import pandas as pd

In [12]:
df = pd.DataFrame(all_books)
df.head()

,title,author_name,first_publish_year
0,The Lord of the Rings,J.R.R. Tolkien,1954
1,The Two Towers,J.R.R. Tolkien,1954
2,The Fellowship of the Ring,J.R.R. Tolkien,1954
3,The Return of the King,J.R.R. Tolkien,1950
4,Novels (Hobbit / Lord of the Rings),J.R.R. Tolkien,1979


### SQlite Database to store retrieved information

In [14]:
import sqlite3
import os

# Ensure the directory exists
os.makedirs('database', exist_ok=True)

conn = sqlite3.connect("database/sample.db")
cursor = conn.cursor()

table_name = "books"

# if_exists='replace' will drop the table and recreate it each time
df.to_sql(table_name, conn, if_exists='replace', index=False)

print(f"Data successfully loaded into the '{table_name}' table in sample.db")


read_df = pd.read_sql(f"SELECT * FROM {table_name} LIMIT 5", conn)
display(read_df)


conn.close()

Data successfully loaded into the 'books' table in sample.db


,title,author_name,first_publish_year
0,The Lord of the Rings,J.R.R. Tolkien,1954
1,The Two Towers,J.R.R. Tolkien,1954
2,The Fellowship of the Ring,J.R.R. Tolkien,1954
3,The Return of the King,J.R.R. Tolkien,1950
4,Novels (Hobbit / Lord of the Rings),J.R.R. Tolkien,1979
